[Reference](https://towardsdatascience.com/creating-automated-python-dashboards-using-plotly-datapane-and-github-actions-ff8aa8b4e3)

In [1]:
datapane_api_token = "" #@param {type:"string"}
datapane_server_url = "https://datapane.com" #@param {type:"string"}

In [2]:
!pip3 install --quiet datapane 

import datapane as dp
dp.login(token=datapane_api_token, server=datapane_server_url) # Make sure you've set your API token above!

In [4]:
import pandas as pd
import numpy as np
import plotly.figure_factory as ff
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import datetime as dt

import plotly.io as pio
pio.templates.default = "ggplot2"

In [5]:
# Using offline data
zm = pd.read_csv("https://raw.githubusercontent.com/datapane/gallery/master/stock-reporting/Zoom_stock_data.csv")
nflx = pd.read_csv("https://raw.githubusercontent.com/datapane/gallery/master/stock-reporting/netflix_stock_data.csv")

In [10]:
# Using live data, do the following: 
# !pip install pandas_datareader
from pandas_datareader import data

# Only get the adjusted close.
zm = data.DataReader("ZM", 
                       start='2019-1-1', 
                       end=dt.datetime.today(), 
                       data_source='yahoo').reset_index()

nflx = data.DataReader("NFLX", 
                       start='2019-1-1', 
                       end=dt.datetime.today(),
                       data_source='yahoo').reset_index()

In [9]:
trace0 = go.Scatter(x=nflx.Date, y=nflx.Close, name='nflx')
fig0 = go.Figure([trace0])
fig0.update_layout(
    title={
        'text': "Netflix Stock Price",
        'x':0.5,
        'xanchor': 'center'})
fig0.show()

In [7]:
nflx['10-day MA'] = nflx['Close'].rolling(window=10).mean()
nflx['20-day MA'] = nflx['Close'].rolling(window=20).mean()
nflx['50-day MA'] = nflx['Close'].rolling(window=50).mean()

trace0 = go.Scatter(x=nflx.Date, y=nflx.Close, name='NFLX')
trace1 = go.Scatter(x=nflx.Date, y=nflx['10-day MA'], name='10-day MA')
trace2 = go.Scatter(x=nflx.Date, y=nflx['20-day MA'], name='20-day MA')
fig1 = go.Figure([trace0, trace1, trace2])
fig1.update_layout(
    title={
        'text': "Netflix Stock Price",
        'x':0.5,
        'xanchor': 'center'})
fig1.show()

In [11]:
trace0 = go.Scatter(x=nflx.Date, y=nflx.Close, name='NFLX', line=dict(color='lime'))
trace1 = go.Scatter(x=zm.Date, y=zm.Close, name='ZM', line=dict(color='red'))

fig4 = go.Figure([trace0, trace1])
fig4.update_layout(dict(
    title=dict({
        'text': "Netflix vs Zoom Stock Price",
        'x':0.5,
        'xanchor': 'center'}),
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=3,
                     label='3m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(count=12,
                     label='12m',
                     step='month',
                     stepmode='backward')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date')))
fig4.show()


In [12]:
trace0 = go.Scatterpolar(
      r = [nflx['Close'].mean(),nflx['Open'].min(), nflx['Low'].min(),nflx['High'].max()],
      theta = ['Close','Open','Low','High'], line =  dict(
            color = 'lime'), name='NFLX',
      fill = 'toself')

trace1 = go.Scatterpolar(
      r = [zm['Close'].mean(),zm['Open'].min(), zm['Low'].min(),zm['High'].max()],
      theta = ['Close','Open','Low','High'], line =  dict(
            color = 'red'), name='ZM',
      fill = 'toself'
    )

fig5 = go.Figure([trace0,trace1])
fig5.update_layout(go.Layout(
      polar = dict(
        radialaxis = dict(
          visible = True)),
            title=dict({
        'text': "Netflix vs Zoom Stock Price (Radar Chart)",
        'x':0.5,
        'xanchor': 'center'})))
fig5.show()

In [13]:
nflx = nflx[['Open','Close','Volume']]
nflx["index"] = np.arange(len(nflx))

fig7 = go.Figure(ff.create_scatterplotmatrix(nflx, diag='box', index='index',size=3,
                               height=600, width=1150, colormap='RdBu',
                                   title={
                                    'text': "Netflix Stock Price (Scatterplot Matrix)",
                                    'x':0.5,
                                    'xanchor': 'center'}))
fig7.show()

In [14]:
import datapane as dp

netflix_today = nflx.iloc[-1] 
zm_today = zm.iloc[-1]

dp.Report(
    dp.Page(
        title='Dashboard',
        blocks=[
            dp.Group(
                dp.BigNumber(
                    heading="Netflix Day Performance",
                    value="${:,.2f}".format(netflix_today.Close),
                    prev_value="${:,.2f}".format(netflix_today.Open),
                ), 
                dp.BigNumber(
                    heading="Zoom Day Performance",
                    value="${:,.2f}".format(zm_today.Close),
                    prev_value="${:,.2f}".format(zm_today.Open),
                ),        
                columns=2 
            ),
            dp.Group(
                fig0, fig1, fig2, fig3, fig4, fig5,
                columns=2, 
            ),
            fig7
      ]),
      dp.Page(title='Datasets', blocks=[
          "_These datasets are pulled live when the report is created_.",                  
          dp.Select(
              dp.DataTable(nflx, label='Netflix'), 
              dp.DataTable(zm, label='Zoom')
          )
      ])
).upload(name='Stock Market Report', open=True)